In [ ]:
from ipywidgets import Button, VBox, HBox, Label
from threading import Thread
import time
from ipycanvas import MultiCanvas, hold_canvas

# Canvas Bildgrösse
canvas_width, canvas_height = 400, 300
paddle_width, paddle_height = 10, 60
ball_radius = 5
ball_speed = [5, 5]  # Initial ball speed; x and y
paddle_speed = 20
player_paddle_y = canvas_height / 2 - paddle_height / 2
computer_paddle_y = canvas_height / 2 - paddle_height / 2
ball_position = [canvas_width / 2, canvas_height / 2]
player_score = 0
computer_score = 0
 
# Intiliasiiere MultiCanvas
multi_canvas = MultiCanvas(3, width=canvas_width, height=canvas_height)
background_layer, game_layer, ui_layer = multi_canvas
 
# Hintergrund
background_layer.fill_style = 'purple'
background_layer.fill_rect(0, 0, canvas_width, canvas_height)
 
# Label Widgets für score Anzeige
player_score_display = Label(value=f"Player: {player_score}")
computer_score_display = Label(value=f"Computer: {computer_score}")
game_over_display = Label(value="")
 
# Computer Logik
computer_hit_counter = 0  # Counter für erfolgreiche Treffer
 
def draw():
    with hold_canvas(game_layer):
        game_layer.clear()
        game_layer.fill_style = 'white'
        # Player paddleong_game_lukas_melsingen_tis-tel-22-zh
        game_layer.fill_rect(0, player_paddle_y, paddle_width, paddle_height)
        # Computer paddle
        game_layer.fill_rect(canvas_width - paddle_width, computer_paddle_y, paddle_width, paddle_height)
        # Ball
        game_layer.fill_circle(ball_position[0], ball_position[1], ball_radius)
    # Punkte aktualisieren
    player_score_display.value = f"Player: {player_score}"
    computer_score_display.value = f"Computer: {computer_score}"
def restart_game(_=None):
    global player_paddle_y, computer_paddle_y, player_score, computer_score, ball_position, ball_speed, computer_hit_counter
    player_paddle_y = canvas_height / 2 - paddle_height / 2
    computer_paddle_y = canvas_height / 2 - paddle_height / 2
    player_score = 0
    computer_score = 0
    ball_position = [canvas_width / 2, canvas_height / 2]
    ball_speed = [5, 5]  # Reset ball Geschwindigkeit
    computer_hit_counter = 0  # Reset abgewehrte Bälle
    game_over_display.value = ""  # Game Over Message löschen
    with hold_canvas(ui_layer):
        ui_layer.clear()  # UI Layer löschen
    draw()  # Neus Canhvas Board
    Thread(target=game_loop).start()  # Game Loop Neustart

def game_loop():
    global ball_position, player_paddle_y, computer_paddle_y, player_score, computer_score, ball_speed, computer_hit_counter
    while True:
        if player_score >= 5:
            with hold_canvas(ui_layer):
                ui_layer.clear()
                ui_layer.font = '48px serif'
                ui_layer.fill_text('You Win!', 100, canvas_height // 2)
            break  # Spiel fertig, Spieler gewinnt
        elif computer_score >= 5:
            with hold_canvas(ui_layer):
                ui_layer.clear()
                ui_layer.font = '34px serif'
                ui_layer.fill_text('Game Over! You Lose!', 35, canvas_height // 2)
            break  # Spiel fertig, Computer gewinnt
        # Ball Geschwindigkeit
        ball_position[0] += ball_speed[0]
        ball_position[1] += ball_speed[1]
        # Ball Kollision mit Wänden
        if ball_position[1] <= ball_radius or ball_position[1] >= canvas_height - ball_radius:
            ball_speed[1] = -ball_speed[1]
        # Ball Kollisio mit Paddles
        if ((ball_position[0] <= paddle_width + ball_radius and
             player_paddle_y < ball_position[1] < player_paddle_y + paddle_height) or
            (ball_position[0] >= canvas_width - paddle_width - ball_radius and
             computer_paddle_y < ball_position[1] < computer_paddle_y + paddle_height)):
            ball_speed[0] = -ball_speed[0]
            if ball_position[0] >= canvas_width - paddle_width - ball_radius:
                computer_hit_counter += 1  # Computer Treffer
        if ball_position[0] < 0:
            computer_score += 1
            ball_position = [canvas_width / 2, canvas_height / 2]
            ball_speed = [5, 5]  # Reset Ball Geschwindigkeit
            computer_hit_counter = 0
        elif ball_position[0] > canvas_width:
            player_score += 1
            ball_position = [canvas_width / 2, canvas_height / 2]  # Reset Position
            ball_speed = [-5, 5]  # Reset Ball Geschwindigkeit
            computer_hit_counter = 0  # Reset Tefferer Zähler
        # Computer Paddle Bewegung
        if ball_speed[0] > 0:
            if computer_hit_counter % 3 != 2:  # Jedes 3. mal verfehlt der Computer den Ball
                if computer_paddle_y < ball_position[1] - paddle_height / 2:
                    computer_paddle_y = min(computer_paddle_y + paddle_speed, canvas_height - paddle_height)
                else:
                    computer_paddle_y = max(computer_paddle_y - paddle_speed, 0)
            else:
                # Paddle wird nicht bewegt vom Computer
                pass
        draw()
        time.sleep(0.05)  # Spielgeschwindigkeit
ong_game_lukas_melsingen_tis-tel-22-zh
def move_player_up(_):
    global player_paddle_y
    player_paddle_y = max(player_paddle_y - paddle_speed, 0)
    draw()
def move_player_down(_):
    global player_paddle_y
    player_paddle_y = min(player_paddle_y + paddle_speed, canvas_height - paddle_height)
    draw()

# Buttons zum bewegen
button_player_up = Button(description="Up")
button_player_down = Button(description="Down")
button_player_up.on_click(move_player_up)
button_player_down.on_click(move_player_down)
player_controls = VBox([button_player_up, button_player_down, player_score_display])
computer_controls = VBox([computer_score_display])

# Neustart Knopf
restart_button = Button(description="Restart / Speed up")
restart_button.on_click(restart_game)

# Anzeige Buttons
display(HBox([player_controls, multi_canvas, computer_controls]))

# Game Loop in sepparatem Thread
Thread(target=game_loop, daemon=True).start()

# Neustart Knopf
display(restart_button)